# Sharks are attacking!

### ¡Importación de datos!

Importamos las librerías que vamos a necesitar.

In [1]:
import pandas as pd
import numpy as np

Importamos los datos en formato csv con los que vamosa a trabajar.

In [2]:
data = pd.read_csv('shark-attacks.csv', encoding='ISO-8859-1')

Aplicamos un **.head()** para poder visualizar el dataframe.

In [3]:
data.head(10)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


Mostramos todas las columnas que hay en el dataframe.

In [4]:
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

Usamos un **.describe** para analizar la moda de cada variable y así sacar una primera conclusión visual.

In [5]:
data.describe(include = 'O').transpose()

,count,unique,top,freq
Case Number,5992,5976,2006.09.02,2
Date,5992,5128,1957,11
Type,5992,6,Unprovoked,4386
Country,5949,203,USA,2116
Area,5590,785,Florida,990
Location,5496,3929,"New Smyrna Beach, Volusia County",157
Activity,5465,1492,Surfing,904
Name,5792,5009,male,495
Sex,5425,6,M,4835
Age,3311,151,17,148


### ¡Limpieza de datos!

A simple vista podemos comprobar que hay columnas con muchos datos nulos, por lo que vamos a ver si efectivamente hay columnas con un número significativo de Nulls y en ese caso las eliminamos de nuestro análisis.

In [6]:
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

Analizando las variables con un mayor número de datos nulos, considero que la variable edad y time (una vez limpiadas) pueden ser significativas, por lo que partiré eliminando las variables Unnamed que están vacías.

In [7]:
drop_cols = list(null_cols[null_cols > 3500].index)
data = data.drop(drop_cols, axis=1)

In [8]:
data.head(10)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,N,NaN,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,N,15h15,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,N,14h30,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,Y,15h40,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,N,Late afternoon,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984


Con **.info** obtenemos la información del dataset, el número de columnas, el número de datos dentro de cada columna y qué tipo de datos tiene.

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 22 columns):
Case Number               5992 non-null object
Date                      5992 non-null object
Year                      5992 non-null int64
Type                      5992 non-null object
Country                   5949 non-null object
Area                      5590 non-null object
Location                  5496 non-null object
Activity                  5465 non-null object
Name                      5792 non-null object
Sex                       5425 non-null object
Age                       3311 non-null object
Injury                    5965 non-null object
Fatal (Y/N)               5973 non-null object
Time                      2779 non-null object
Species                   3058 non-null object
Investigator or Source    5977 non-null object
pdf                       5992 non-null object
href formula              5991 non-null object
href                      5989 non-null object
C

Elimino las columnas que sé que no me van a interesar en absoluto, ya sea porque dan los mismos valores o porque son valores que no influyen en el dataset y reducimos nuestro dataset de 22 a 13 columnas.

In [10]:
before = len(data.columns)
select_columns = ['Case Number', 'Date', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order']

data = data.drop(select_columns, axis=1)
after = len(data.columns)
print('Número de columnas finales: ', after)

Número de columnas finales:  13


In [11]:
data.head(10)

,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN
1,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN
2,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN
3,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN
4,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark
5,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,NaN,Shark rammed boat. No injury to occupant,N,NaN,NaN
6,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,60s,Minor injury to arm,N,15h15,3' to 4' shark
7,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,51,Severe lacerations to shoulder & forearm,N,14h30,"Tiger shark, 10?"
8,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,50,FATAL,Y,15h40,NaN
9,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,12,Lacerations & punctures to lower right leg,N,Late afternoon,NaN


Mostramos los nulos de la variable 'Sex'.

In [12]:
null_displ = data[(data['Sex '].isnull()==True)]
null_displ = null_displ[['Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ']]
null_displ

,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
5,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,NaN,Shark rammed boat. No injury to occupant,N,NaN,NaN
29,2016,Boat,USA,Alabama,8 miles off Mobile,Fishing in Alabama Deep Fishing Rodeo,Occupant: Ben Raines,NaN,NaN,"No injury, shark bit trolling motor",N,NaN,"Tiger shark, 10'"
81,2016,Invalid,USA,California,"Bolsa Chica State Park, Orange County",Surfing,unknown,NaN,NaN,Board reportedly bumped by shark. No injury,N,Morning,Shark involvement not confirmed
109,2015,Boat,AUSTRALIA,New South Wales,Lake Macquarie,Fishing,6 m boat: occupants Stephen & Andrew Crust,NaN,NaN,"No injury, shark rammed boat & bit motor",N,10h30,"White shark, 3.5 m"
225,2015,Boat,ATLANTIC OCEAN,NaN,NaN,Transatlantic Rowing,"Avalon, a carbon kevlar monohull: 8 occupants",NaN,NaN,"No injury, shark bit rudder",N,NaN,White shark
241,2014,Unprovoked,AUSTRALIA,New South Wales,Bherwerre Beach,Surfing,Jeff Brown,NaN,NaN,Lacerations to both feet,N,Morning,2 m shark
243,2014,Sea Disaster,GREECE,NaN,33 nautical miles off Othonoi Island,NaN,Passenger ferry Norman Atlantic,NaN,NaN,"Of 9 bodies recovered, one was bitten by a shark",N,NaN,Shark involvement prior to death still to be d...
244,2014,Invalid,SOUTH AFRICA,KwaZulu-Natal,Durban,Swimming,"5 people claimed to have been injured by a ""ba...",NaN,NaN,Minor cuts on feet,N,Morning,Shark involvement not confirmed & highly unlikely
254,2014,Boat,USA,California,"Franklin Point, San Mateo County",NaN,Boat: occupants: Matt Mitchell & 2 other people,NaN,NaN,"Shark bumped boat, no injury to occupants",N,NaN,White shark
285,2014,Invalid,USA,Florida,"Atlantic Beach, Duval County",NaN,child,NaN,NaN,Shark involvement not confirmed,N,16h18,Shark involvement not confirmed


Mostramos los elementos que conforman la variable 'Sex' y cambiamos los nombres a las columnas 'Sex' y 'Species' para eliminarles los espacios que tenían.

In [13]:
data['Sex '].value_counts()

M      4835
F       585
M         2
lli       1
.         1
N         1
Name: Sex , dtype: int64

In [14]:
data = data.rename(columns={'Sex ':'Sex',
                            'Species ':'Species'})
data.columns

Index(['Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name',
       'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species'],
      dtype='object')

Dado que no nos interesa ni transformar los nulos ni los valores que no son Mujer, creamos una variable nueva **filtered** para filtrar los datos que tenemos acordes a la Variable Sexo-Mujer(F).

In [15]:
filtered = data[(data['Sex']=='F')]

filtered.head(10)

,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
7,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,51,Severe lacerations to shoulder & forearm,N,14h30,"Tiger shark, 10?"
9,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,12,Lacerations & punctures to lower right leg,N,Late afternoon,NaN
30,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,female,F,9,Minor injury to leg,N,1300,NaN
31,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",NaN,female,F,11,Minor injury to toes,N,11h00,NaN
34,2016,Unprovoked,SPAIN,Canary Islands,"Las Teresitas, Tenerife",Wading,female,F,10,"5 tiny puncture marks to lower leg, treated wi...",N,NaN,Angel shark
38,2016,Unprovoked,USA,Florida,"Melbourne Beach, Brevard County",Swimming,female,F,42,"Buttocks, thigh, left hand & wrist injured",N,14h30 / 15h30,NaN
48,2016,Unprovoked,USA,Texas,"Pirates Beach, Galveston",Floating in tube,Marin Alice Melton,F,6,Injury to lower leg,N,17h30,3' to 4' shark
52,2016,Unprovoked,AUSTRALIA,Western Australia,Mindarie,Diving,Doreen Collyer,F,60,FATAL,Y,11h30,3+ m shark
57,2016,Unprovoked,USA,California,"Corona Del Mar, Newport, Orange County",Swimming,Maria Korcsmaros \n,F,52,Injuries to arm and shoulder,N,16h00,NaN
59,2016,Unprovoked,USA,Florida,"Vero Beach, Indian River County",Swimming,Mary Marcus,F,57,Puncture wounds to thigh,N,12h00,NaN
